In [1]:
import numpy as np
import pandas as pd
import os
import sklearn
import skopt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold, train_test_split

from utils import load_training_dataset, setup_logging
from sklearn.pipeline import Pipeline

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
import pandas as pd


from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


In [2]:
fudan_filepath = 'data/Yang_PRJNA763023/Yang_PRJNA763023_SE/parsed/normalized_results/'

In [5]:
def load_tsv_files(folder):
    # Initialize an empty dictionary to store the variables
    var_dict = {}
    
    # Iterate over the files in the folder
    for file in os.listdir(folder):
        # Only consider files with the '.tsv' extension
        if file.endswith('.tsv'):
            # Load the data into a pandas DataFrame
            df = pd.read_csv(os.path.join(folder, file), sep='\t') # skiprows=1
            if len(df.columns) == 1:
                df = pd.read_csv(os.path.join(folder, file), sep='\t', skiprows=1)
                df = df.transpose()
                df.columns = df.iloc[0]
                df.drop(index=df.index[0], axis=0, inplace=True)
                #print(file)
            else:
                #print(file)
                columnNames = df.columns.tolist()
                firstColName = columnNames[0]
                df.set_index(firstColName, inplace=True)
            # Get the file name without the '.tsv' extension
            name = os.path.splitext(file)[0]
            
            # Assign the DataFrame to a variable with the file name
            globals()[name] = df
            
            # Add the variable to the dictionary with the file name as the key
            var_dict[name] = globals()[name]
    
    # Return the dictionary of variables
    return var_dict

In [6]:
def load_data(filepath):

    data_files = load_tsv_files(filepath)

    for key, value in data_files.items():
        globals()[key] = value

In [7]:
load_data(fudan_filepath)

In [8]:
global_vars = globals()

In [9]:
file_names = list(("pielou_e_diversity", "simpson_diversity", "phylum_relative", "observed_otus_diversity", "family_relative",
"class_relative", "fb_ratio", "enterotype", "genus_relative", "species_relative", "shannon_diversity", "domain_relative",
"order_relative", "simpson_e_diversity"))

In [10]:
def load_metadata(filepath):
    data = pd.read_csv(filepath, sep=",", usecols=["Run", "disease_stat"])
    data.set_index('Run', inplace=True)
    return data

In [11]:
yang_metadata = load_metadata("data/Yang_PRJNA763023/metadata.csv")

In [12]:
def preprocess_data(file_name):
    dataset = globals()[file_name]
    #print(file_name)
    data = dataset.join(yang_metadata)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

    for col in X.columns:
        #find max value of column
        max_value_train = np.nanmax(X[col][X[col] != np.inf])
        #replace inf and -inf in column with max value of column 
        X[col].replace([np.inf, -np.inf], max_value_train, inplace=True)
        #drop the inf values from the test set
        X = X.replace([np.inf, -np.inf], np.nan).dropna()
    #get the respective y when we drop observations from the test set
    y = y[y.index.isin(X.index)]

    #labelEncoder
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    #y_test = le.transform(y_test)
    
    
        
    return X, y #X_train, X_test, y_train, y_test

def rf_classification(file_name):
    X_train, X_test, y_train, y_test = preprocess_data(file_name)
    clf = RandomForestClassifier(n_estimators= 200, max_depth=40, random_state=1234, class_weight={1: 1})
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    #nu = np.unique(y_test, return_counts=True)
    p_r_f1_support = precision_recall_fscore_support(y_test, y_pred, average='macro')
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('The results for ' + file_name + " are: accuracy " + str(acc) + ", conf.mat.:" + str(cm) + ", roc auc:" + str(roc_auc) + ", precision, recall, f1:" + str(p_r_f1_support))
    return y_pred

In [ ]:
# Set up logging
logger = setup_logging("tune_random_forest")

for file_name in file_names:
    X, y = preprocess_data(file_name)

    #tune random forest
    estimator = Pipeline([
        #("preprocessor", MinMaxScaler()),
        ("model", RandomForestClassifier(random_state=1234))
    ])

    # search space
    search_space = {
        "model__n_estimators": Integer(10, 100),
        "model__max_depth": Categorical([None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]),
        "model__min_samples_split": Integer(2, 20),
        "model__min_samples_leaf": Integer(1, 5),
        "model__max_features": Categorical([None, "sqrt", "log2"]),
    }

    # cross validation strategy
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1234)

    for score in ["accuracy", "recall"]:
        # optimizer
        optimizer = BayesSearchCV(
            estimator=estimator,
            search_spaces=search_space,
            scoring=score,
            cv=cv,
            n_jobs=-1,
            verbose=1,
            random_state=1234
        )

        # fit
        optimizer.fit(X, y)


        # log best score and parameters
        #TODO enter file_name
        logger.info(f"File: {file_name}")
        logger.info(f"Scoring: {score}")
        logger.info(f"Best score: {optimizer.best_score_}")
        logger.info(f"best params: {optimizer.best_params_}")
        df = pd.DataFrame(optimizer.cv_results_)[["params", "mean_test_score", "std_test_score", "rank_test_score"]]
        logger.info("CV results:")
        logger.info(df.sort_values("rank_test_score").to_string())
        logger.info("\n")


NumExpr defaulting to 8 threads.


Fitting 50 folds for each of 1 candidates, totalling 50 fits


/home/andrea/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for eac

/home/andrea/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


...

Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits


/home/andrea/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits


Scoring: accuracy
Best score: 0.5260056683986774
best params: OrderedDict([('model__max_depth', 20), ('model__max_features', 'log2'), ('model__min_samples_leaf', 1), ('model__min_samples_split', 2), ('model__n_estimators', 10)])
CV results:
                                                                                                                                                params  mean_test_score  std_test_score  rank_test_score
47     {'model__max_depth': 20, 'model__max_features': 'log2', 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 10}         0.526006        0.041234                1
28     {'model__max_depth': 20, 'model__max_features': 'log2', 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 10}         0.526006        0.041234                1
29       {'model__max_depth': 30, 'model__max_features': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 17}      

Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for eac

In [48]:
for file_name in file_names:
    rf_classification(file_name)

The results for pielou_e_diversity are: accuracy 0.4840182648401826, conf.mat.:[[59 53]
 [60 47]], roc auc:0.48301902536715624, precision, recall, f1:(0.48289915966386554, 0.48301902536715624, 0.48246439550787373, None)
The results for simpson_diversity are: accuracy 0.5251141552511416, conf.mat.:[[59 53]
 [51 56]], roc auc:0.5250751001335114, precision, recall, f1:(0.5250625521267723, 0.5250751001335113, 0.525025025025025, None)
The results for phylum_relative are: accuracy 0.5844748858447488, conf.mat.:[[75 37]
 [54 53]], roc auc:0.5824849799732977, precision, recall, f1:(0.5851421188630491, 0.5824849799732977, 0.5802388524969986, None)
The results for observed_otus_diversity are: accuracy 0.5525114155251142, conf.mat.:[[64 48]
 [50 57]], roc auc:0.5520694259012016, precision, recall, f1:(0.5521303258145362, 0.5520694259012016, 0.5520537652362665, None)
The results for family_relative are: accuracy 0.730593607305936, conf.mat.:[[85 27]
 [32 75]], roc auc:0.7299315754339119, precision